### Line-by-Line Extraction with PyMuPDF

In [1]:
import fitz
import pandas as pd

In [11]:
file_path = '/Users/yiminglin/Documents/Codebase/Pdf_reverse/data/raw/benchmark1/id_15.pdf'
num_pages = 1

In [3]:
try:
    doc = fitz.open(file_path)
    print(f"Successfully opened: {doc.page_count} pages")
except Exception as e:
    print(f"Error opening the PDF: {e}")

Successfully opened: 253 pages


In [4]:
# This was used for reference when testing the get_phrases() method.
def get_lines(page):
    return page.get_text('lines').split('\n')

In [5]:
def get_words(page):
    return [list(word_tuple) for word_tuple in page.get_text('words')]

In [13]:
def get_phrases(page):
    docs_by_words = get_words(page)
    doc_by_phrases = []
    cur_bbox, prev_line, cur_phrase = None, None, None
    for word_info in docs_by_words[0]:
        bbox, word, line = word_info[:4], word_info[4], word_info[5:7]
        if line == prev_line:
            cur_bbox[2:] = bbox[2:]
            cur_phrase += ' ' + word
        else:
            if prev_line:
                doc_by_phrases.append([cur_phrase]+cur_bbox)
            cur_bbox = bbox
            cur_phrase = word
        prev_line = line
    doc_by_phrases.append([cur_phrase]+cur_bbox)
    return doc_by_phrases

In [14]:
phrases, pages = [], doc.pages(0,num_pages)
for i in range(num_pages):
    phrases += [[i+1]+phrase for phrase in get_phrases(next(pages))]

TypeError: 'float' object is not subscriptable

In [10]:
phrases_df = pd.DataFrame(phrases, columns=['Page', 'Phrase', 'x0', 'y0', 'x1', 'y1'])
phrases_df

,Page,Phrase,x0,y0,x1,y1


In [134]:
phrases_df.to_csv(file_path.split('/')[-1][:-4]+'_phrases.csv')